In [1]:
import os
os.chdir('../')

In [2]:
import pandas as pd

data = pd.read_csv('Datasets/analysis_data.csv')
y=data['monthly_spend']
X=data.drop(labels=['monthly_spend'],axis=1)

numeric_variables=X.select_dtypes('number').columns
categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_columns)
    ],
    remainder='passthrough' # Keep numeric columns as they are
)
X_encoded=preprocessor.fit_transform(X)
feature_names = list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns))
numeric_cols = X.select_dtypes(include=['number']).columns.tolist()
new_cols = feature_names + numeric_cols

X_processed=pd.DataFrame(X_encoded, columns=new_cols)

estimator = RandomForestRegressor(n_estimators=10, random_state=2121)

# 3. Create the Iterative Imputer (MICE)
mice_imputer = IterativeImputer(
    estimator=estimator,
    max_iter=10,        # Number of imputation cycles (T)
    initial_strategy='mean', # Initial placeholder strategy
    random_state=2121
)

# 4. Fit the Imputer and Transform the Data
# The imputer automatically handles all columns, using each as a predictor for the others.
df_imputed_array = mice_imputer.fit_transform(X_processed)

# Convert the resulting NumPy array back to a Pandas DataFrame
df_imputed = pd.DataFrame(df_imputed_array, columns=X_processed.columns)


In [3]:
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler

X_train = df_imputed

# 3. Ridge Regression with automatic alpha selection
ridge = RidgeCV(alphas=[0.1, 1.0, 10.0, 50.0, 100.0], cv=5)
ridge.fit(X_train, y)

# 4. Predictions
y_hat = ridge.predict(X_train)


c:\Users\athar\anaconda4\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.64242e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.63826e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.63944e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.62998e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\linear_model\_ridge.py:21

In [6]:
from sklearn.metrics import root_mean_squared_error

root_mean_squared_error(y,y_hat)

257.3882947777293

PredictionRidge with transformed y=log(y)

In [8]:
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler
import numpy as np

X_train2 = df_imputed
y_train_log = np.log1p(y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# 3. Ridge Regression with automatic alpha selection
ridge2 = RidgeCV(alphas=[0.1, 1.0, 10.0, 50.0, 100.0], cv=5)
ridge2.fit(X_train_scaled, y_train_log)

# 4. Predictions
y_hat2 = np.expm1(ridge2.predict(X_train_scaled))

root_mean_squared_error(y,y_hat2)


294.984210013998

Elastic Net

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error
import numpy as np

# ------------------------------
# 1. Select stepwise features
# ------------------------------
X_train =df_imputed
y_train = y   # DO NOT SCALE y


# ------------------------------
# 3. Fit ElasticNetCV
# ------------------------------
# l1_ratio controls mix of L1 (Lasso) and L2 (Ridge)
# 0.0 = Ridge, 1.0 = Lasso, 0.5 = mix
enet = ElasticNetCV(
    l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9],   # Elastic mixing
    alphas=np.logspace(-3, 2, 20),       # Regularization values
    cv=5,
    random_state=42
)

enet.fit(X_train, y_train)

print("Best alpha:", enet.alpha_)
print("Best l1_ratio:", enet.l1_ratio_)

# ------------------------------
# 4. Predict on training set
# ------------------------------
y_hat = enet.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, y_hat))
print("Elastic Net RMSE:", rmse)


c:\Users\athar\anaconda4\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2114125879.292324, tolerance: 927722.8243141122
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1376394044.0349252, tolerance: 927722.8243141122
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1793548610.3605852, tolerance: 927722.8243141122
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: 

Best alpha: 0.001
Best l1_ratio: 0.7
Elastic Net RMSE: 257.3886262026583


c:\Users\athar\anaconda4\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.326e+09, tolerance: 1.155e+06
  model = cd_fast.enet_coordinate_descent(


: 